In [7]:
from kafka import KafkaConsumer
import sys
import json
import time
from confluent_kafka import Consumer, KafkaError
from json import loads
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime
import warnings

# Suppress sklearn warning about missing feature names
warnings.filterwarnings("ignore", message="X does not have valid feature names, but IsolationForest")

import numpy as np  # Import NumPy for array manipulation
# Reading messages from the consumer with a 5-second interval

In [8]:
# Kafka broker address
bootstrap_servers = 'localhost:9093'
 
# Kafka topic to consume messages from
kafka_topic = 'Topic_KMeans'
 
# # Consumer group ID
# group_id = 'group1'
 
# InfluxDB connection details
influxdb_host = 'localhost'
influxdb_port = 8086
influxdb_username = 'mahirasadzade'
influxdb_password = 'mahira123'
bucket = "BloodPressure"
organization = "OstProject"
influxdb_token = "DG2B6mm_vDI_4oVm9OW3pCiv1A6ID5Jr9XGgn_7uIbG8iElhhzwYOOXw5Begzk3fTI9M1nh-XI6Qb2qA1k7RIg=="

# Create Kafka consumer configuration
consumer_config = {
    'bootstrap.servers': bootstrap_servers,
    # 'group.id': group_id,
    'auto.offset.reset': 'earliest'
}
 
with InfluxDBClient(url="http://localhost:8086", token=influxdb_token, org=organization,bucket=bucket) as client:
        write_api = client.write_api(write_options=SYNCHRONOUS)
        consumer = KafkaConsumer(kafka_topic, bootstrap_servers=['localhost:9093'],auto_offset_reset='earliest', api_version=(0,10),enable_auto_commit=True,value_deserializer=lambda x: loads(x.decode('utf-8')))
        print("Connection established")
        i=0
        for message in consumer:
            print(f"{message.value}")
            # Reshape the input data into a 2D array
            datetime = np.array(message.value.get('DateTime')).reshape(-1, 1)
            sbp = np.array(message.value.get('SBP')).reshape(-1, 1)
            dbp = np.array(message.value.get('DBP')).reshape(-1, 1)
            prediction = np.array(message.value.get('prediction')).reshape(-1, 1)

            #print(sbp[0][0], dbp[0][0], detection)
            row_count = 1
             # Create InfluxDB point
            point = Point("KMeans") \
                    .field("SBP", float(sbp[0][0])) \
                    .field("DBP", float(dbp[0][0])) \
                    .field("prediction", float(prediction[0][0])) \
                    .field("All records", row_count) 
                   
            from tabulate import tabulate
            table = [["DateTime", "SBP", "DBP", "Anomaly"],
                             [datetime, sbp, dbp, prediction]]
            print(tabulate(table, headers="firstrow", tablefmt="grid"))
            
            #print(tabulate(table, headers="firstrow", tablefmt="grid"))
            
            # Write the point to InfluxDB
            write_api = client.write_api(write_options=SYNCHRONOUS)
            write_api.write(bucket, organization, point)
            
            i+=1
            print("Message sent to influxDB", i)
        

Connection established
{'DateTime': '2020-10-24 21:37', 'SBP': 93.0, 'DBP': 51.2, 'prediction': 0}
+------------------------+-------+-------+-----------+
| DateTime               |   SBP |   DBP |   Anomaly |
+========================+=======+=======+===========+
| [['2020-10-24 21:37']] |    93 |  51.2 |         0 |
+------------------------+-------+-------+-----------+
Message sent to influxDB 1
{'DateTime': '2020-10-24 21:38', 'SBP': 88.2, 'DBP': 49.2, 'prediction': 0}
+------------------------+-------+-------+-----------+
| DateTime               |   SBP |   DBP |   Anomaly |
+========================+=======+=======+===========+
| [['2020-10-24 21:38']] |  88.2 |  49.2 |         0 |
+------------------------+-------+-------+-----------+
Message sent to influxDB 2
{'DateTime': '2020-10-24 21:39', 'SBP': 89.6, 'DBP': 49.8, 'prediction': 0}
+------------------------+-------+-------+-----------+
| DateTime               |   SBP |   DBP |   Anomaly |
+========================+=======+=